In [4]:
import glob
import os

In [5]:
class_names = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [5]:
## для аннотаций  

dict_info =  {"description": "ExDark",
              "url": "https://github.com/cs-chan/Exclusively-Dark-Image-Dataset",
        "version": "1.0",
        "year": 2015,
        "contributor": "ExDark",
        "date_created": "2015/01/01"
      }
dict_licenses = [{"url": "https://github.com/cs-chan/Exclusively-Dark-Image-Dataset",
          "id": 1,
          "name": "BSD-3"}]
img ={  "license": 1,
            "file_name": "000000397133.jpg",
            "coco_url": "",
            "height": 427,
            "width": 640,
            "date_captured": "2015",
            "flickr_url": "",
            "id": 397133}
annot= {
            "area": 702.1057499999998,
            "iscrowd": 0,
            "image_id": 289343,
            "bbox": [473.07,395.93,38.65,28.67],
            "category_id": 18,
            "id": 1768
       }
dict_cat = [
        {"id": 1,"name": "person"},
        {"id": 2,"name": "bicycle"},
        {"id": 3,"name": "car"},
        {"id": 4,"name": "motorbike"},
        {"id": 9,"name": "boat"}, 
        {"id": 16,"name": "cat"},
        {"id": 40,"name": "bottle"},
        {"id": 42,"name": "cup"},
        {"id": 53,"name": "dog"},
        {"id": 57,"name": "chair"},
        {"id": 61,"name": "diningtable"}]

name_key ={'Bicycle': 2,
   'Boat': 9,
   'Bottle': 40,
   'Bus': 6,
   'Car': 3,
   'Cat': 16,
   'Chair': 57,
   'Cup': 42,
   'Dog': 53,
   'Motorbike': 4,
   'People': 1,
   'Diningtable': 61}

In [6]:
name_key.keys()

dict_keys(['Bicycle', 'Boat', 'Bottle', 'Bus', 'Car', 'Cat', 'Chair', 'Cup', 'Dog', 'Motorbike', 'People', 'Diningtable'])

In [7]:
from PIL import Image 
import os
import re
import json
from shutil import copyfile

#copyfile(src, dst)

## Кодировать в анотацию сосо

#### СМ.  http://cocodataset.org/#format-data

In [10]:

count_img = 0
count_annot_all = 0
result = []

imgs = []
val_imgs = []
annots = []
val_annots = []
for top, dirs, files in os.walk('./ExDark'):
    #print(top, dirs, files)
    #print(top)
    valid_count = 0
    for nm in files:     
        if nm.endswith(".txt"):
            with open(top+'/'+nm) as f:
                fn = re.findall('(.+)\.txt', top+'/'+nm)[0]
                new_name = re.sub('\.(jpg|png|JPEG|JPG|jpeg)','',nm)
                #copyfile(src, dst)
                img_id_arr = re.findall('\d{4}_(\d{5})\.(?:jpg|png|JPEG|JPG|jpeg)\.txt',nm)
                try:
                    img_id=int(img_id_arr[0])
                    img = Image.open(fn)
                    width,height = img.size
                except Exception:
                    print('ошибка на файле', fn)
                    continue  
                #print('/home/user/netology/netology_git/yolov3_pytorch/notebooks'+fn[1:])    
                new_img ={  "license": 1,"file_name": '/home/user/netology/netology_git/yolov3_pytorch/notebooks'+fn[1:],"coco_url": "",
                    "height": height,"width": width,"date_captured": "2015","flickr_url": "","id": img_id} 
                if valid_count < 75:
                    val_imgs.append(new_img)
                else:    
                    imgs.append(new_img)
                count_img +=1    
                str_ann = f.readline()
                count_annot = 0
                for i in f:
                    res = re.findall('(\w+) (\d+) (\d+) (\d+) (\d+) .*', i)
                    
                    if len(res)<1:
                        print('ошибка в аннотации:', i, ' в файле:', fn )
                        continue
                    type_obj, x, y, w, h = res[0]
                    #print(img_id, type(int(img_id)))
                    if type_obj in name_key.keys():
                        new_annot ={ "area": int(w) * int(h),
                                    "iscrowd": 0,
                                    "image_id": int(img_id)*100,
                                    "bbox": [int(x), int(y), int(w), int(h)],
                                    "category_id": name_key[type_obj],
                                    "id": img_id*100+count_annot
                                   }
                        if valid_count < 75:
                            val_annots.append(new_annot)
                        else:
                            annots.append(new_annot)
                        count_annot+=1
                valid_count += 1        
                count_annot_all = count_annot_all + count_annot       
                        
                        
print(count_img,count_annot_all)                    

result =  {
    "info" : dict_info, 
    "images" : imgs, 
    "annotations" : annots, 
    "licenses" : dict_licenses,
}  
val_result =  {
    "info" : dict_info, 
    "images" : val_imgs, 
    "annotations" : val_annots, 
    "licenses" : dict_licenses,
}  

with open('ExDarc_annot.json', 'w') as f:
    f.write(json.dumps(result))
with open('ExDarc_annot_val.json', 'w') as f:
    f.write(json.dumps(val_result))                    

ошибка на файле ./ExDark/Bus/2015_02330.jpg
ошибка на файле ./ExDark/Chair/2015_04274.jpg
ошибка на файле ./ExDark/Dog/2015_05189.jpg
7360 22215


In [ ]:

   # bicycle - bicycle - 2
   # boat - boat - 9
   # bottle - bottle - 40
  #  bus - bus - 6
  #  car - car - 3
  #  cat - cat - 16
  #  chair - chair - 57
 #   cup - cup - 42
  #  dog - dog - 53
 #   motorbike - motorbike 4
 #   people - person - 1
 #   table - diningtable - 61


In [ ]:
from PIL import Image 
import os
import re
import json
from shutil import copyfile

## Кодировать в анотоцию VOC
####  каждый файл с картинокой имеет описание    {{filename}}.jpg -> {{filename}}.txt
  
####  со строками: тип_объекта x y width hieght
      
#### например: 1 150 100 100

In [12]:
count_img = 0
count_annot_all = 0
result = []

imgs = []
val_imgs = []
annots = []
val_annots = []
for top, dirs, files in os.walk('.'):
    #print(top, dirs, files)
    #print(top)
    valid_count = 0
    for nm in files:     
        if nm.endswith(".txt"):
            fn = re.findall('(.+)\.txt', nm)[0]
            
            #print(fn,len(re.findall(r'\.(jpg|png|JPEG|JPG|jpeg)',fn)))
            if len(re.findall(r'\.(jpg|png|JPEG|JPG|jpeg)',fn)) < 1:
                continue
            try:
                
                img = Image.open(top+'/'+fn)
                width,height = img.size
            except Exception:
                print('ошибка на файле', top+'/'+fn)
                continue     
                
            new_name = re.sub('\.(jpg|png|JPEG|JPG|jpeg)','',nm)
            dest_p = './train_img/' if valid_count > 50 else './val_img/'
            #print(fn,new_name,dest_p)
            with open(top+'/'+nm) as f:
                arr = f.readlines()
                del arr[0]
                for i in range(len(arr)):
                    arr[i] = arr[i].lower()
                    arr[i] = arr[i].replace('people','person')  
                    #arr[i] = arr[i].replace('diningtable','table')
                    arr[i] = arr[i].replace('person','1')
                    arr[i] = arr[i].replace('bicycle','2')
                    arr[i] = arr[i].replace('car','3')
                    arr[i] = arr[i].replace('motorbike','4')
                    arr[i] = arr[i].replace('bus','6')
                    arr[i] = arr[i].replace('boat','9') 
                    arr[i] = arr[i].replace('cat','16')
                    arr[i] = arr[i].replace('dog','17')
                    arr[i] = arr[i].replace('bottle','40')
                    arr[i] = arr[i].replace('cup','42')
                    arr[i] = arr[i].replace('chair','57')
                    arr[i] = arr[i].replace('table','61')
                    m_arr = arr[i].split(' ')[:5]
                    m_arr[1] = str(int(m_arr[1])/width)
                    m_arr[2] = str(int(m_arr[2])/height)
                    m_arr[3] = str(int(m_arr[3])/width)
                    m_arr[4] = str(int(m_arr[4])/height)
                    #print(m_arr)
                    arr[i] = ' '.join(m_arr)
                    
                     
                #print(dest_p+new_name, "\n".join(arr))
                with open(dest_p+new_name, 'w') as f:
                    f.write("\n".join(arr))
            #copyfile(top+'/'+nm, dest_p+new_name)
            
            copyfile(top+'/'+fn, dest_p+fn)
            valid_count += 1 
            if top != "./ExDark/People" and valid_count > 200:
                break
                
                
                        




In [49]:
import numpy as np

np.loadtxt('./train_img/2015_00254.txt')

array([  2., 111., 174., 317., 499.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.])